环形数组技巧利用求模（余数）运算，将普通数组变成逻辑上的环形数组，可以让我们用 O(1) 的时间在数组头部增删元素。

In [ ]:
from typing import TypeVar, Generic, Optional, List

T = TypeVar("T")


class CircleArray(Generic[T]):
    """环形数组主要是为了高效的两端操作，尤其是当需要循环利用内存空间"""

    def __init__(self, size=1):
        self.size = size
        self.arr: List[Optional[T]] = [None] * size
        self.start = 0
        self.end = 0
        self.count = 0

    def resize(self, newSize):
        """扩容

        Args:
            newSize (_type_): 新的大小
        """
        new_arr: List[Optional[T]] = [None] * newSize
        for i in range(self.count):
            new_arr[i] = self.arr[(self.start + i) % self.size]
        self.arr = new_arr
        self.start = 0
        self.end = self.count
        self.size = newSize

    def increase_size_if_need(self):
        """如果数组满了则扩容为原来的两倍"""
        if self.is_full():
            self.resize(self.size * 2)

    def decrease_size_if_need(self):
        """如果数组元素数量减少到原大小的四分之一，则减小数组大小为一半"""
        if self.count > 0 and self.count == self.size // 4:
            self.resize(self.size // 2)

    def add_first(self, val):
        self.increase_size_if_need()
        self.start = (self.start - 1 + self.size) % self.size
        self.arr[self.start] = val
        self.count += 1

    def remove_first(self):
        if self.is_empty():
            return
        self.arr[self.start] = None
        self.start = (self.start + 1) % self.size
        self.count -= 1
        self.decrease_size_if_need()

    def add_last(self, val):
        self.increase_size_if_need()
        self.end = (self.end + 1 + self.size) % self.size
        self.arr[self.end] = val
        self.count += 1

    def remove_last(self):
        if self.is_empty():
            return
        # 环形数组是左闭右开，即start指向的是第一个元素，但end指向的是最后一个元素的下一个位置
        # 因此这里删除的话要先左移，再来赋值
        self.end = (self.end - 1) % self.size
        self.arr[self.end] = None
        self.count -= 1
        self.decrease_size_if_need()

    def get_first(self):
        if self.is_empty():
            return
        return self.arr[self.start]

    def get_last(self):
        if self.is_empty():
            return
        return self.arr[self.end]

    def is_full(self) -> bool:
        return self.count == self.size

    def size(self) -> int:
        return self.count

    def is_empty(self) -> bool:
        return self.count == 0